<a href="https://colab.research.google.com/github/hailong09/fake_news_detection_model_finetuning/blob/main/fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install tensorflow transformers datasets scikit-learn pandas

In [11]:
from datasets import load_dataset
import pandas as pd


# Load Fake news dataset
dataset = load_dataset("liar")


Dataset({
    features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
    num_rows: 10269
})
           id  label  ... pants_on_fire_counts              context
0   2635.json      0  ...                  0.0             a mailer
1  10540.json      1  ...                  0.0      a floor speech.
2    324.json      2  ...                  9.0               Denver
3   1123.json      0  ...                 44.0       a news release
4   9028.json      1  ...                  2.0  an interview on CNN

[5 rows x 14 columns]


In [22]:
from transformers import TFBertForSequenceClassification, BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)  # 2 classes: Fake/Real

# Tokenize function
def tokenize_func(example):
  return tokenizer(example["statement"], padding="max_length", truncation=True)

# Apply tokenization
dataset = dataset.map(tokenize_func, batched=True)

# Convert labels (modify based on dataset)
# def process_labels(examples):
#     examples["label"] = [0 if l == 0 else 1 for l in examples["label"]]  # Fake = 0, Real = 1
#     return examples

# dataset = dataset.map(process_labels)
df_train = pd.DataFrame(dataset["train"])



All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/10269 [00:00<?, ? examples/s]

Map:   0%|          | 0/1283 [00:00<?, ? examples/s]

Map:   0%|          | 0/1284 [00:00<?, ? examples/s]

In [23]:
import tensorflow as tf

# Convert Dataset to TF tensors
def convert_to_tf_dataset(split):
  features = {key: tf.convert_to_tensor(dataset[split][key]) for key in ["input_ids", "attention_mask"]}
  labels = tf.convert_to_tensor(dataset[split]["label"])
  return tf.data.Dataset.from_tensor_slices((features, labels)).batch(16)

# Create TensorFlow datasets
train_dataset = convert_to_tf_dataset("train")
valid_dataset = convert_to_tf_dataset("validation")
test_dataset = convert_to_tf_dataset("test")


In [ ]:
# Compile and Fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

# Train the model
model.fit(train_dataset, validation_data=valid_dataset, epochs=3)

Epoch 1/3
